Generate and save corpus

In [5]:

import generate_corpus
from datasets import load_dataset
import pandas as pd
import sentencepiece as spm
import torch
import random

#load dataset
dataset = load_dataset("ms_marco", "v1.1")
train_df = dataset['train']
train_df = pd.DataFrame(train_df)
val_df = dataset['validation']
val_df = pd.DataFrame(val_df)
test_df = dataset['test']
test_df = pd.DataFrame(test_df)
#concat all dataframe to generate corpus from entire dataset
dataset_df = pd.concat([train_df,val_df,test_df],axis = 0)
#generate corpus
corpus = generate_corpus.generate_corpus(dataset_df)

#save corpus
file_path = "corpus.txt"

# Open the file for writing
with open(file_path, "w",encoding="utf-8") as file:
    # Iterate through the list and write each element to the file
    for item in corpus:
        file.write(item + "\n")

# Close the file
file.close()
print('corpus.txt is created and saved')


tokenizer:
* Generate sentencepiece .model and .vocab for future tokenizing if called as main
* Tokenize otherwise

In [ ]:
import sentencepiece as spm

class sp_tokenizer():
    def __init__(self, sp_model, text):
        self.sp_model = sp_model
        self.sp_tokenize(text)
    
    def sp_tokenize(self,text):
        sp = spm.SentencePieceProcessor()
        sp.load(self.sp_model)
        tokens = sp.encode_as_pieces(text)
        token_ids = [sp.piece_to_id(token) for token in tokens]
        return token_ids
        



if __name__ == '__main__':
    # Set the paths for your input text file and output model and vocab files
    input_file = 'corpus.txt'
    model_file = 'ms_marco_model.model'
    vocab_file = 'ms_marco_vocab.txt'

    # Train the SentencePiece model
    spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_file, vocab_size=16000)
    print('sentencepiece model and vocab files are saved')


Generate word embeddings via CBOW

In [2]:
import load_corpus

corpus = load_corpus.load_corpus()




In [3]:
import generate_CBOW_ds

cbow = generate_CBOW_ds.W2VData(corpus, window_size=2)

In [35]:

dl = torch.utils.data.DataLoader(cbow.data, batch_size=4, shuffle=True)

In [38]:
import torch
import pandas
import tqdm
import sentencepiece as spm
import model_cbow

sp = spm.SentencePieceProcessor()
sp.load('ms_marco_model.model.model')

# Get the vocabulary size
vocab_size = sp.get_piece_size()

cbow = model_cbow.CBOW(vocab_size, 16)
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(cbow.parameters(), lr=0.001)


for epoch in range(2):
  total_loss = 0
  for context, target in tqdm.tqdm(dl, desc=f"Epoch {epoch+1}/10", unit="batch"):
    optimizer.zero_grad()
    log_probs = cbow(context)
    loss = loss_function(log_probs, target)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1}/10, Loss: {total_loss}")
  torch.save(cbow.state_dict(), f"./cbow_epoch_{epoch+1}.pt")

Epoch 1/10:   0%|          | 0/19438719 [00:03<?, ?batch/s]


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list